In [608]:
import logging
from collections import namedtuple
import random
from matplotlib import pyplot as plt
import numpy
import sys

In [609]:
N = 100
PROBLEM_SIZE = N**2             # può avere senso mettere anche qua N, perché se il gene è la lista, si possono avere N**2 liste per ogni fitter ma 
                                # questo significa che possono essere prese tutte? Beh si, ma non avrebbe senso perché vogliamo il minor numero di 
                                # liste prese.
                                # Possiamo allora inizare considerando comunque N**2 elementi del problema, in ogni posto abbiamo 1 se prendiamo la
                                # lista con quell'indice, 0 se non la prendiamo
POPULATION_SIZE = N*2            # aumentare la popolazione sembra avere un effetto positivo
OFFSPRING_SIZE = int(N*1.5)         

NUM_GENERATIONS = 1000          # Più elementi ci sono e più generazioni prendono

TOURNAMENT_SIZE = int(N/3)
MIN_NUMBER = sys.float_info.min
GENETIC_OPERATOR_RANDOMNESS = 0.3
logging.getLogger().setLevel(logging.INFO)

In [610]:
def problem(N, seed=42):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [611]:
GOAL={i for i in range(N)}
print(GOAL)
list_of_lists = problem(N)
tmp =  {tuple(x) for x in list_of_lists}    # rimozione delle inner list ripetute
list_of_lists = list(tmp)                   
print(list_of_lists)
PROBLEM_SIZE = len(list_of_lists)           # può avere senso ridurre la dimensione del genoma in questo senso
print(PROBLEM_SIZE)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99}
[(0, 1, 4, 7, 10, 12, 13, 15, 19, 23, 25, 32, 38, 42, 44, 45, 51, 52, 55, 58, 59, 62, 65, 67, 78, 79, 84, 92, 98), (4, 5, 6, 13, 18, 21, 24, 31, 36, 37, 38, 49, 50, 54, 56, 57, 58, 59, 66, 67, 68, 71, 78, 80, 82, 88, 90, 94, 97, 99), (2, 3, 5, 7, 10, 11, 14, 17, 24, 25, 28, 35, 39, 41, 45, 50, 51, 57, 62, 65, 67, 68, 70, 74, 77, 79, 82, 85, 86, 98), (4, 5, 12, 14, 17, 19, 27, 30, 35, 38, 47, 49, 52, 53, 56, 58, 59, 62, 63, 68, 69, 75, 78, 83, 85, 88, 89, 93, 95), (0, 2, 4, 10, 11, 16, 18, 20, 22, 23, 26, 30, 34, 36, 38, 44, 48, 52, 53, 57, 60, 62, 63, 66, 75, 76, 78, 80, 81, 83, 88, 90, 91, 98, 99), (0

In [612]:
Individual = namedtuple("Individual", ["genome", "fitness"])

def gen2List(genome):
    list = []
    for i, g in enumerate(genome):
        if g:
            list += list_of_lists[i]
    return list

def tournament(population, tournament_size=TOURNAMENT_SIZE):          
    return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness) 

def one_cut_cross_over(g1, g2):                      
    cut = random.randint(0, PROBLEM_SIZE)
    return g1[:cut] + g2[cut:]

def uniform_cross_over(g1, g2):
    new_genoma = []
    for i in range(PROBLEM_SIZE):
        if i%2:
            new_genoma.append(g1[i])
        else:
            new_genoma.append(g2[i])
    return tuple(new_genoma)

def mutation(g):                                
    point = random.randint(0, PROBLEM_SIZE - 1)      
    return g[:point] + (1 - g[point],) + g[point + 1 :] 

def print_population(population):
    for j, ind in enumerate(population):
        list = []
        s_temp = set()
        for i, g in enumerate(ind[0]):
            if g:
                list.append(list_of_lists[i])
                for x in list_of_lists[i]:
                    s_temp.add(x)
        print(f"{j+1} : {list} -> {s_temp} - {ind[1]}")


def compute_fitness(genome):                                # eventualmente tenere conto della lunghezza per il calcolo della fitness
    list = gen2List(genome)
    repetitions = len(list) - len(set(list))
    g = len(GOAL - set(list))
    # if repetitions == 0:
    #     repetitions = MIN_NUMBER
    # return (N - len(GOAL - s_temp))/(repetitions + 1)           # Questa variante introduce troppa penalità se si hanno ripetizioni 
    # return (N - len(GOAL - set(list))) * (len(list)) / (repetitions + 1)
       
    # return (N - len(GOAL - set(list))) + 1 / (repetitions + 1)
    # return (N - len(GOAL - set(list))) - numpy.sqrt(len(list))          # besty

    return (N - len(GOAL - set(list))) - pow(len(list), 1/2)
    # return (N - len(GOAL - set(list))) - numpy.log2(len(list)+1)

    # return (N - len(GOAL - set(list))) + (1 / (repetitions + 1))**N  
    # return N - len(GOAL - set(list)) + (1/(repetitions + 1))**1/(N - len(GOAL - set(list)))
    # return (N - len(GOAL - set(list))) + repetitions / (len(list) + 1)       
    # return (N - len(GOAL - set(list))) + 1/( (repetitions) / (len(list) + 1) + 1 )

    # return (N - len(GOAL - set(list))) - numpy.exp(repetitions) / (len(list) + 1)**N        
    # return (N - len(GOAL - set(list))) + numpy.exp(repetitions) / ((len(GOAL - set(list)) + (repetitions)) + 1)

    # return (N - len(GOAL - set(list))) - (repetitions)**3  / (len(list) + 1)         
    # return (N - len(GOAL - set(list))) + 1/((repetitions)**(len(list)) +1 ) 


# def compute_fitness(genome):                               
#     list = gen2List(genome)
#     repetitions = len(list) - len(set(list))
#     if repetitions == 0:
#         repetitions = MIN_NUMBER
#     length = len(list)
#     normalized_cost = repetitions/(length) 
#     return normalized_cost
    
def all_equal(list):
    return list[:-1] == list[1:]

def check_converge_goal(population):
        l_genome = []
        for ind in population:
            l_genome.append(gen2List(ind[0]))
        if set(gen2List(ind[0])) == GOAL and all_equal(l_genome):
            return True

def check_goal(genoma):
    if set(gen2List(genoma)) == GOAL:
        return True
    else:
        return False

def compute_w(genome):
    list = []
    for i, g in enumerate(genome):
        if g:
            list += list_of_lists[i]
    return len(list)
    
#CROSS_OVER IMPLEMENTATION
def cross_over(g1, g2): #prendo due genomi e restituisco un individuo con caratteristiche 50% di g1 e 50% di g2
    #print("parenti scelti", g1, g2)
    cut = random.randint(0, len(list_of_lists) - 1)
    #print("taglio sull'indice", cut)
    #print("padre1", g1[:cut])
    #print("padre2", g2[cut:])
    #print("genoma dopo cross over")
    #print(g1[:cut] + g2[cut:])
    if sum(g1[cut:])> sum(g2[cut:]):
        f1=g1[cut:] 
    else: 
        f1=g2[cut:]
    if sum(g1[:cut])> sum(g2[:cut]):
        f2=g1[:cut] 
    else: 
        f2=g2[:cut]

    return f1 + f2

In [613]:
population = list()

# Non so se questa cosa è fattibile, ma possiamo fare in modo che la probabilità di prendere una lista sia bassa
# oppure partire da una sola lista (quindi un solo valore di 1, perché senno può trovarlo anche a culo)

# for genome in [tuple([1 if random.random() < 0.3 else 0 for _ in range(PROBLEM_SIZE)]) for _ in range(POPULATION_SIZE)]:
#     population.append(Individual(genome, compute_fitness(genome))) 
    
for genome in [tuple(0 for _ in range(PROBLEM_SIZE)) for _ in range(POPULATION_SIZE)]:
    genome = mutation(genome)                       # ho chiamato "mutation" ma solo perché volevo fare la stessa cosa
    population.append(Individual(genome, compute_fitness(genome))) 

logging.info(f"init: pop_size={len(population)}; max={max(population, key=lambda i: i.fitness)[1]}")
# print_population(population)

INFO:root:init: pop_size=200; max=35.51925930159214


In [614]:
fitness_log = [(0, i.fitness) for i in population]  

best_fit = 0
for g in range(NUM_GENERATIONS):
    offspring = list()
    for i in range(OFFSPRING_SIZE):
        if random.random() < GENETIC_OPERATOR_RANDOMNESS:                         
            p = tournament(population)                  
            o = mutation(p.genome)                    
        else:                                          
            p1 = tournament(population)                 
            p2 = tournament(population)
            o = uniform_cross_over(p1.genome, p2.genome)            
        f = compute_fitness(o)                                      
        fitness_log.append((g + 1, f))                     
        offspring.append(Individual(o, f))                 
    population += offspring      
    population = sorted(population, key=lambda i: i[1], reverse=True)[:POPULATION_SIZE]
    # print(f"gen: {g}")
    # print_population(population)
    
    if best_fit < population[0][1] and check_goal(population[0][0]):
        best_fit = population[0][1]
        w = len(gen2List(population[0][0]))
        gen = g

In [615]:
print(f"conv_fit {best_fit} found at {gen} with w : {w}")
# off_line = [max(f[1] for f in fitness_log if f[0] == x) / (x + 1) for x in range(NUM_GENERATIONS)]      
# on_line = [max(f[1] for f in fitness_log if f[0] <= x) / (x + 1) for x in range(NUM_GENERATIONS)]       
# gen_best = [max(f[1] for f in fitness_log if f[0] == x) for x in range(NUM_GENERATIONS)]

# plt.figure(figsize=(15, 6))
# sulle x abbiamo le generazioni, sulla y la fitness
# la linea in alto segue quelli che hanno la fitness più alta per ogni generazione
# le linee in basso fanno capire l'andamento per ogni generazione
# plt.scatter([x for x, _ in fitness_log], [y for _, y in fitness_log], marker=".")
# plt.plot([x for x, _ in enumerate(gen_best)], [y for _, y in enumerate(gen_best)])
# plt.plot([x for x, _ in enumerate(on_line)], [y for _, y in enumerate(on_line)])
# plt.plot([x for x, _ in enumerate(off_line)], [y for _, y in enumerate(off_line)])

conv_fit 85.54316770519904 found at 134 with w : 209
